In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Zheryu\Anaconda3\envs\image-processing\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ifft']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### Import

In [1]:
import numpy as np
import pyfftw.interfaces.scipy_fftpack as spfft
import scipy.signal as spsignal
import skimage.transform as sktransform
import matplotlib.pyplot as plt

In [2]:
from filter_tools import *
from util import *
from pyramid_tools import *

In [3]:
    


def phase_amplify(video, 
                  magnification_factor, 
                  fl, fh, fs, 
                  attenuate_other_frequencies=False, 
                  pyramid_type="octave", 
                  sigma=0, 
                  temporal_filter=fir_window_bp
                 ):
    num_frames, h, w, num_channels = video.shape
    pyr_height = max_scf_pyr_height((h, w))

    if pyr_type is "octave":
        print("Using vanilla octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 4)
    elif pyr_type is "halfOctave":
        print("Using half octave pyramid")
        filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 8, t_width=0.75)
    elif pyr_type is "smoothHalfOctave":
        print("Using smooth half octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=2)
    elif pyr_type is "quarterOctave":
        print("Using quarter octave pyramid.")
        filters = get_filters_smooth_window((h, w), 8, filters_per_octave=4)
    else:
        print("Invalid filter type. Specify ocatave, halfOcatave, smoothHalfOctave, or quarterOctave")
        return None
    print('Number of filters: ', len(filters))
    
    yiq_video = np.zeros((num_frames, h, w, num_channels))
    fft_video = np.zeros((num_frames, h, w), dtype=complex64)

    for i in range(num_frames):
        yiq_video[i] = rgb2yiq(video[i])
        fft_video[i] = spfft.fftshift(spfft.fft2(yiq_video[i][:,:,0]))

    magnified_y_channel = np.zeros((num_frames, h, w), dtype=complex64)
    dc_frame_index = 0

    
    for i in range(1,len(filters)-1):
        print("processing level "+str(i))

        dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
        dc_frame_no_mag = dc_frame / np.abs(dc_frame)
        dc_frame_phase = np.angle(dc_frame)

        total = np.zeros(fft_video.shape, dtype=float)
        filtered = np.zeros(fft_video.shape, dtype=complex64)

        for j in range(num_frames):
            filtered[j] = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[j]))
            total[j] = simplify_phase(np.angle(filtered[j]) - dc_frame_phase)

        print("bandpassing...")
        total = temporal_filter(total, fl/fs, fh/fs).astype(float)

        for j in range(num_frames):
            phase_of_frame = total[j]
            if sigma != 0:
                phase_of_frame = amplitude_weighted_blur(phase_of_frame, np.abs(filtered[j]), sigma)

            phase_of_frame *= magnification_factor

            if attenuate_other_frequencies:
                temp_orig = np.abs(filtered[j])*dc_frame_no_mag
            else:
                temp_orig = filtered[j]
            magnified_component = 2*filters[i]*spfft.fftshift(spfft.fft2(temp_orig*np.exp(1j*phase_of_frame)))

            magnified_y_channel[j] = magnified_y_channel[j] + magnified_component

    for i in range(num_frames):
            magnified_y_channel[i] = magnified_y_channel[i] + (fft_video[i]*(filters[-1]**2))

    out = np.zeros(yiq_video.shape[:-1])
    print(out.shape)

    for i in range(num_frames):
        # color
        out_frame  = np.dstack((np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i]))), yiq_video[i,:,:,1:3]))
        out[i] = yiq2rgb(out_frame)
        # monochrome
        # out[i] = out_frame
        # out_frame  = np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i])))

    return out.clip(min=0, max=1)


<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_15376/321223522.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if pyr_type is "octave":
/tmp/ipykernel_15376/321223522.py:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "halfOctave":
/tmp/ipykernel_15376/321223522.py:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "smoothHalfOctave":
/tmp/ipykernel_15376/321223522.py:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pyr_type is "quarterOctave

### Load the resources

In [3]:
video_file = 'IMG_0339_ext.MOV'
video = load_video(video_file)
video.shape

The frame size for reading (1080, 1920) is different from the source frame size (1920, 1080).


(243, 1920, 1080, 3)

In [21]:
load_video?

Signature: load_video(filename)
Docstring: <no docstring>
File:      ~/code/optimaize/magnifier_miscelaneous_code/video_motion_magnification_pyramid_tools/util.py
Type:      function


In [18]:
magnification_factor = 50
fl = 12
fh = 18
fs = 240
attenuate_other_frequencies=False # False
pyr_type = "octave"
sigma = 5 # 5
temporal_filter =  iir_butter #fir_window_bp, iir_butter


#### Resize our video

In [15]:
# video2 = np.zeros((len(video), 200, 200, 3))
# for i in range(len(video)):
#     video2[i] = sktransform.resize(video[i], (200,200))

# RUN

In [19]:
video_out = phase_amplify(
    video[200:260,:,:,:], 
    magnification_factor, 
    fl, 
    fh, 
    fs, 
    attenuate_other_frequencies=attenuate_other_frequencies, 
    pyramid_type=pyr_type, 
    sigma=sigma, 
    temporal_filter=temporal_filter
)

Using vanilla octave pyramid
Number of filters:  30
processing level 1
bandpassing...
Using bandpass filter (butter).
processing level 2
bandpassing...
Using bandpass filter (butter).
processing level 3
bandpassing...
Using bandpass filter (butter).
processing level 4
bandpassing...
Using bandpass filter (butter).
processing level 5
bandpassing...
Using bandpass filter (butter).
processing level 6
bandpassing...
Using bandpass filter (butter).
processing level 7
bandpassing...
Using bandpass filter (butter).
processing level 8
bandpassing...
Using bandpass filter (butter).
processing level 9
bandpassing...
Using bandpass filter (butter).
processing level 10
bandpassing...
Using bandpass filter (butter).
processing level 11
bandpassing...
Using bandpass filter (butter).
processing level 12
bandpassing...
Using bandpass filter (butter).
processing level 13
bandpassing...
Using bandpass filter (butter).
processing level 14
bandpassing...
Using bandpass filter (butter).
processing level 15

In [20]:
fps = 30
video_out_name = 'IMG_0368_out_octave_50x_12-18Hz_5sig_iir_butter_bp_grayscale.mp4'
write_video(video_out, fps, video_out_name)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1344, 670) to (1344, 672) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior 

# TESTS

In [3]:
video_file = 'IMG_0339_ext.mp4'
video = load_video(video_file)
video = video[0:10,:100,:100,:]
video.shape

The frame size for reading (1080, 1920) is different from the source frame size (1920, 1080).


(10, 100, 100, 3)

In [5]:
num_frames, h, w, num_channels = video.shape
pyr_height = max_scf_pyr_height((h, w))

filters = get_filters((h, w), 2**np.array(list(range(0,-pyr_height-1,-1)), dtype=float), 4)
    
yiq_video = np.zeros((num_frames, h, w, num_channels))
fft_video = np.zeros((num_frames, h, w), dtype=complex64)

for i in range(num_frames):
    yiq_video[i] = rgb2yiq(video[i])
    fft_video[i] = spfft.fftshift(spfft.fft2(yiq_video[i][:,:,0]))

print('video')
print(video[0,:5,:5,0])
print('y channel frame 0')
print(yiq_video[0,:5,:5,0])
print('q channel frame 0')
print(yiq_video[0,:5,:5,1])
print('i channel frame 0')
print(yiq_video[0,:5,:5,2])

print('fft_video')
print('frame 0')
print(fft_video[0,:3,:3])
print('frame 2')
print(fft_video[2,:3,:3])

print()
print('Filters applied to first frame')
dc_frame_index = 0
for i in range(len(filters)):
    dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
    # dc_frame_no_mag = dc_frame / np.abs(dc_frame)
    dc_frame_phase = np.angle(dc_frame)
    print(i)
    print(dc_frame[:3,:3])
    print()
    
print('')
print('Test phase (lo level):')
print(dc_frame_phase[:3,:3])
print('Test phase (11 level):')
dc_frame = spfft.ifft2(spfft.ifftshift(filters[11]*fft_video[dc_frame_index]))    
# dc_frame_no_mag = dc_frame / np.abs(dc_frame)
dc_frame_phase = np.angle(dc_frame)
print(dc_frame_phase[:3,:3])
print()

i = 9 # filter number
print(f'Test simplify_phase and phase difference on channel {i}')
total = np.zeros(fft_video.shape, dtype=float)
filtered = np.zeros(fft_video.shape, dtype=complex64)
dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
dc_frame_phase = np.angle(dc_frame)
for frame in range(10): # llop on frames
    filtered[frame] = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[frame]))
    total[frame] = simplify_phase(np.angle(filtered[frame]) - dc_frame_phase)
    print(f'frame {frame}')
    print(total[frame][:3,:3])
print()
print('total[0][:3,:3]')
print(total[0][:3,:3])
print('total[3][:3,:3]')
print(total[3][:3,:3])
print()

print(f'Test temporal filter on channel {i}')
temporal_filter =  iir_butter
fl = 12
fh = 18
fs = 240
total = temporal_filter(total, fl/fs, fh/fs).astype(float)
total = temporal_filter(total, fl/fs, fh/fs).astype(float)
print('First frame')
print(total[0][:3,:3])
print('Second frame')
print(total[1][:3,:3])
print('9- frame')
print(total[9][:3,:3])
print('Site [7,13] along time axis')
for t in total:
    print(t[7,13],end=' ')
break




magnified_y_channel = np.zeros((num_frames, h, w), dtype=complex64)
dc_frame_index = 0

    
for i in range(1,len(filters)-1):
    print("processing level "+str(i))

    dc_frame = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[dc_frame_index]))    
    dc_frame_no_mag = dc_frame / np.abs(dc_frame)
    dc_frame_phase = np.angle(dc_frame)

    total = np.zeros(fft_video.shape, dtype=float)
    filtered = np.zeros(fft_video.shape, dtype=complex64)

    for j in range(num_frames):
        filtered[j] = spfft.ifft2(spfft.ifftshift(filters[i]*fft_video[j]))
        total[j] = simplify_phase(np.angle(filtered[j]) - dc_frame_phase)

    print("bandpassing...")
    total = temporal_filter(total, fl/fs, fh/fs).astype(float)

    for j in range(num_frames):
        phase_of_frame = total[j]
        if sigma != 0:
            phase_of_frame = amplitude_weighted_blur(phase_of_frame, np.abs(filtered[j]), sigma)

        phase_of_frame *= magnification_factor

        if attenuate_other_frequencies:
            temp_orig = np.abs(filtered[j])*dc_frame_no_mag
        else:
            temp_orig = filtered[j]
        magnified_component = 2*filters[i]*spfft.fftshift(spfft.fft2(temp_orig*np.exp(1j*phase_of_frame)))

        magnified_y_channel[j] = magnified_y_channel[j] + magnified_component

for i in range(num_frames):
        magnified_y_channel[i] = magnified_y_channel[i] + (fft_video[i]*(filters[-1]**2))

out = np.zeros(yiq_video.shape[:-1])
print(out.shape)

for i in range(num_frames):
    # color
    out_frame  = np.dstack((np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i]))), yiq_video[i,:,:,1:3]))
    out[i] = yiq2rgb(out_frame)
    # monochrome
    # out[i] = out_frame
    # out_frame  = np.real(spfft.ifft2(spfft.ifftshift(magnified_y_channel[i])))



video
[[82 84 86 88 89]
 [82 84 86 88 89]
 [82 84 86 88 89]
 [82 84 86 88 89]
 [82 84 86 88 89]]
y channel frame 0
[[0.28864706 0.2964902  0.30433333 0.31217647 0.31609804]
 [0.28864706 0.2964902  0.30433333 0.31217647 0.31609804]
 [0.28864706 0.2964902  0.30433333 0.31217647 0.31609804]
 [0.28864706 0.2964902  0.30433333 0.31217647 0.31609804]
 [0.28864706 0.2964902  0.30433333 0.31217647 0.31609804]]
q channel frame 0
[[0.03326655 0.03326655 0.03326655 0.03326655 0.03326655]
 [0.03326655 0.03326655 0.03326655 0.03326655 0.03326655]
 [0.03326655 0.03326655 0.03326655 0.03326655 0.03326655]
 [0.03326655 0.03326655 0.03326655 0.03326655 0.03326655]
 [0.03326655 0.03326655 0.03326655 0.03326655 0.03326655]]
i channel frame 0
[[0.00180276 0.00180276 0.00180276 0.00180276 0.00180276]
 [0.00180276 0.00180276 0.00180276 0.00180276 0.00180276]
 [0.00180276 0.00180276 0.00180276 0.00180276 0.00180276]
 [0.00180276 0.00180276 0.00180276 0.00180276 0.00180276]
 [0.00180276 0.00180276 0.00180276 

SyntaxError: 'break' outside loop (3580968011.py, line 83)

In [24]:
rgb2yiq?

Signature: rgb2yiq(img)
Docstring: <no docstring>
File:      ~/code/optimaize/magnifier_miscelaneous_code/video_motion_magnification_pyramid_tools/util.py
Type:      function
